In [13]:
import subprocess
from astropy.table import Table
import os
import csv
#in the furture make packages and import common functions
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from pyraf import gwm
from astropy.io import ascii
import numpy as np
from astropy.stats import sigma_clip
from astropy.io import fits

In [14]:
from pyraf import iraf
from iraf import obsutil
from iraf import psfmeasure
from iraf import phot
from iraf import daofind
from iraf import daophot
from iraf import psf
from iraf import artdata
from iraf import mkobjects
from iraf import starlist

In [15]:
imagename = '/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits'
diffImage = '/home/mj1e16/Simages/diff_44_1.fits'
SImage= '/home/mj1e16/Simages/testsimage.fits'
ext = 44

In [16]:
outDir = '/home/mj1e16/iraf/outputs/' # for iraf output files ONLY
imageDir = '/home/mj1e16/Simages/irafImages/' # for images generated during process ONLY
simageDir = '/home/mj1e16/Simages/' # directory for simulated images
irafDir = '/home/mj1e16/iraf/' #Directory containing iraf installation

In [17]:
def psfmodel(imagename,ext,section,verb=90,thresh=100000):
    
    daoout = '{}_{}daofindone'.format(ext,section)
    photout = '{}_{}photone'.format(ext,section)
    psfout = '{}_{}psfone'.format(ext,section)
    groupout = '{}_{}groupfileone'.format(ext,section)
    opstout = '{}_{}optsone'.format(ext,section)
    
    daofind(imagename+'[{}]'.format(ext),output=daoout,sigma=1.0,threshold=thresh)
    phot(imagename+'[{}]'.format(ext),skyfile=daoout,coords=daoout,output=photout,interactive='no')
    psf(imagename+'[{}]'.format(ext),photfile=daoout,pstfile=photout,psfimage=psfout,opstfile=opstout,groupfile=groupout,interactive='no')
    mkobjects('/home/mj1e16/Simages/diff_44_1.fits', output='/home/mj1e16/Simages/testmkObjects.fits',objects='starfield.dat',gain=110,rdnoise=127,star=psfout)

In [18]:
def mkobjectspluspsf(imagename,diffimage,diffPlusSim,ext,section,Xmax=1000,Xmin=0,Ymax=1000,Ymin=1000,thresh=50000,nsegs=5,goodpsf=''):
    
    daoout = outDir+'{}_{}daofindone'.format(ext,section)
    photout = outDir+'{}_{}photone'.format(ext,section)
    psfout = outDir+'{}_{}psfone'.format(ext,section)
    groupout = outDir+'{}_{}groupfileone'.format(ext,section)
    opstout = outDir+'{}_{}optsone'.format(ext,section)
    starfieldsect = outDir+'starfield_{}.dat'.format(section)
    imseg = imageDir+'imageseg{}.fits'.format(section)
    
    hdu = fits.open(imagename)
    print(imagename)
    image = hdu[ext].section[Xmin:Xmax,Ymin:Ymax]
    hdu = fits.PrimaryHDU(image)
    hdu.writeto(imseg,clobber=True)
    
    with open(starfieldsect,'w') as f:
        f.write("")
    
    daofind(imseg,output=daoout,sigma=1.0,threshold=thresh)
    phot(imseg,skyfile=daoout,coords=daoout,output=photout,interactive='no')
    psf(imseg,photfile=daoout,pstfile=photout,psfimage=psfout,opstfile=opstout,groupfile=groupout,interactive='no')
    starlist(starfieldsect,nstars=5000/nsegs,xmin=Xmin,xmax=Xmax,ymin=Ymin,ymax=Ymax)
    try:
        mkobjects(diffimage,output=diffPlusSim,objects=starfieldsect,gain=110,rdnoise=127,star=psfout+'.fits')
        
    except:
        try:
            #subprocess.call(['rm','/home/mj1e16/Simages/testsimage*'])
            mkobjects(diffimage,output=diffPlusSim,objects=starfieldsect,gain=110,rdnoise=127,star=goodpsf)
            print('Waring: Section {} with bad PSF, {} used inplace'.format(section,goodpsf))
        except:
            print('ERROR: PSF for Section {} should be made manually'.format(section))
    
    return psfout+'.fits'

In [19]:
def findPSFandMkObjects(imageName,diffImage,SImage,extension,imageshape=[1025., 1095.],pixelsize=3.98):
    
    print(imageName)
    
    os.chdir(irafDir)
    
    xlength = imageshape[1]
    ylength = imageshape[0]
    nsegs = 4
    
    xsegment = xlength/nsegs
    ysegment = ylength/nsegs
    PSF = []
    goodpsf=''
    for xsegs in range(nsegs):
        lowx = int(xsegs*xsegment)+15
        highx = int(lowx+xsegment)+15
        psf = []
        for ysegs in range(0,nsegs):
            lowy = int(ysegs*ysegment)+15
            highy = int(lowy+ysegment)+15
            
            
            # stars are being lost from images in xseg=0
            SImage= imageDir+'testsimage_{}_{}.fits'.format(xsegs,ysegs)
            
            goodpsf = mkobjectspluspsf(imageName,diffImage,SImage,extension,'{}_{}'.format(xsegs,ysegs),Xmax=highx,Xmin=lowx,Ymax=highy,Ymin=lowy,nsegs=nsegs**2,goodpsf=goodpsf)
            
            b = diffImage.find('_alt')
            if b == -1:
                diffImage = diffImage[:-5]+'_alt.fits'
            try:
                hdu = fits.open(SImage) #,ignore_missing_end=True)
                imdata = hdu[0].data
                hdu = fits.PrimaryHDU(imdata)
                hdu.writeto(diffImage,clobber=True)
                print('find me')
                print(SImage)
                print(diffImage)
            except:
                print('bad at section {} {}'.format(xsegs,ysegs))

            

            
    #return PSF

In [20]:
def cleanDirectories(dirlist):
    # clean iraf output and input images
    # make sure to have no needed files in these directories
    for directory in dirlist:
        dirfiles = os.listdir(directory)
        for eachFile in dirfiles:
            #print('rm',directory+eachFile)
            subprocess.call(['rm',directory+eachFile])

In [21]:
def stitchStarList(nsegs):
    with open(outDir+'starfield_0_0.dat') as f:
        lines = f.readlines()
    bigstring = ['# X_POS Y_POS MAG\n']
    bigstring.extend(lines[0:18])
    for x in range(nsegs):
        for y in range(nsegs):
            with open(outDir+'starfield_{}_{}.dat'.format(x,y)) as f:
                lines = f.readlines()
            bigstring.extend(lines[18:])


    with open('/home/mj1e16/keplerPhotometry/starlistFull.dat','w') as f:
        f.write(''.join(bigstring))


In [22]:
cleanDirectories([outDir,imageDir]) # cleans iraf output files, will remove all files from these directories

In [26]:
findPSFandMkObjects(imagename,diffImage,SImage,ext)

/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Image: /home/mj1e16/Simages/irafImages/imageseg0_0.fits  fwhmpsf: 2.5  ratio: 1.  theta: 0.  nsigma: 1.5

   52.01    24.52   -1.259   0.691   0.323   0.558     1
  148.02    97.33   -1.209   0.644   0.469   1.000     2
   47.79   121.86   -1.202   0.458   0.250   0.333     3
  220.81   142.56   -0.204   0.782  -0.087   0.276     4
   26.47   145.63   -1.268   0.684  -0.378   0.015 

find me
/home/mj1e16/Simages/irafImages/testsimage_0_0.fits
/home/mj1e16/Simages/diff_44_1_alt.fits
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Image: /home/mj1e16/Simages/irafImages/imageseg0_1.fits  fwhmpsf: 2.5  ratio: 1.  theta: 0.  nsigma: 1.5

  121.58    39.08   -0.886   0.918  -0.267  -0.216     1
  247.86    61.44   -0.919   0.675  -0.162  -0.034     2
   41.48    77.97   -0.237   0.929  -0.271  -0.295     3
  263.99    83.34   -1.116   0.796   0.183   0.303     4
  151.

Killing IRAF task `mkobjects'


Waring: Section 0_1 with bad PSF, /home/mj1e16/iraf/outputs/44_0_0psfone.fits used inplace
find me
/home/mj1e16/Simages/irafImages/testsimage_0_1.fits
/home/mj1e16/Simages/diff_44_1_alt.fits
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Image: /home/mj1e16/Simages/irafImages/imageseg0_2.fits  fwhmpsf: 2.5  ratio: 1.  theta: 0.  nsigma: 1.5

   37.78    51.15   -0.111   0.973  -0.115  -0.016     1
  198.80    63.44   -0.365   0.875   0.376   0.226     2
    7.99    83.34   -1.116   

Killing IRAF task `mkobjects'
Killing IRAF task `mkobjects'


...

Waring: Section 2_2 with bad PSF, /home/mj1e16/iraf/outputs/44_2_1psfone.fits used inplace
find me
/home/mj1e16/Simages/irafImages/testsimage_2_2.fits
/home/mj1e16/Simages/diff_44_1_alt.fits
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Image: /home/mj1e16/Simages/irafImages/imageseg2_3.fits  fwhmpsf: 2.5  ratio: 1.  theta: 0.  nsigma: 1.5

  240.19     4.40   -0.676   0.851  -0.181   0.205     1
  181.18    32.88   -0.689   0.550   0.110   0.395     2
    4.29    50.72   -0.985   

Killing IRAF task `mkobjects'
Killing IRAF task `mkobjects'


ERROR: PSF for Section 2_3 should be made manually
find me
/home/mj1e16/Simages/irafImages/testsimage_2_3.fits
/home/mj1e16/Simages/diff_44_1_alt.fits
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Image: /home/mj1e16/Simages/irafImages/imageseg3_0.fits  fwhmpsf: 2.5  ratio: 1.  theta: 0.  nsigma: 1.5

  134.22    46.04   -1.210   0.818   0.089   0.091     1
   50.62   135.77   -0.147   0.917   0.071  -0.163     2
   25.03   145.54   -0.117   0.979   0.129   0.226     3
   84.45   1

	X:  191.67 Y:  204.21  Mag:  -0.439  Dmin: -3.522321  Dmax: 185850.4
Star 10 has been added to the PSF star list
	X:  186.33 Y:  205.82  Mag:  -0.413  Dmin: -2.46201  Dmax: 185850.4

10 PSF stars read from /home/mj1e16/iraf/outputs/44_3_1photone


Fitting function gauss    norm scatter: 0.9422228

Analytic PSF fit 
    Function: gauss  X: 135.  Y: 116.5  Height: 411118.4  Psfmag: 10.921
    Par1: 3.445434  Par2: 4.576446

Computed 1 lookup table(s)

Writing PSF image /home/mj1e16/iraf/outputs/44_3_1psfone.fits
Writing output PSF star list /home/mj1e16/iraf/outputs/44_3_1optsone
Writing output PSF star group file /home/mj1e16/iraf/outputs/44_3_1groupfileone
find me
/home/mj1e16/Simages/irafImages/testsimage_3_1.fits
/home/mj1e16/Simages/diff_44_1_alt.fits
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standa

Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts

imageseg3_3.fits     23.33     9.91  178.3479   11.631  ok
imageseg3_3.fits     44.76    19.48  179.9086   11.126  ok
imageseg3_3.fits    168.94    36.61  172.5043   10.265  ok
imageseg3_3.fits    139.51    37.75  169.1525   11.466  ok
imageseg3_3.fits     17.96    40.07  181.3425   10.279  ok
imageseg3_3.fits     12.05    48.58  182.4664   11.003  ok
imageseg3_3.fits    227.40    52.12  176.2806   11.171  ok
imageseg3_3.fits      5.72    59.10  181.7462   11.044  ok
imageseg3_3.fits    154.52    73.63  170.3513   10.906  ok
imageseg3_3.fits    212.61    99.98   172.841   11.735  ok
imageseg3_3.fits    155.36   204.58  108.5566   10.986  ok

Analytic psf function(s) (gauss): 	Analytic psf function(s): gauss
Order of variable psf (0): 	Order of variable psf: 0
Psf radius in scale units (11.): 	New psf radius:

In [27]:
stitchStarList(4)